In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 Youtube analysis

* [Google Gen AI SDKs](https://cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview)
  * The new Google Gen AI SDK provides a unified interface to Gemini 2.0 through both the Gemini Developer API and the Gemini API on Vertex AI. With a few exceptions, code that runs on one platform will run on both.
  * This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

* [google-genai 0.3.0](https://pypi.org/project/google-genai/)
* [Google Gen AI SDK](https://googleapis.github.io/python-genai/)
* [python-genai : Github](https://github.com/googleapis/python-genai)

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [1]:
%pip install --upgrade --user --quiet google-genai

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [3]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-2.0-flash-exp" # @param {type:"string"}

### Import libraries

In [4]:
import base64

from google import genai
from google.genai.types import (
    Content,
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleSearch,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    VertexAISearch,
)


### Initalize client

* [genai.Client](https://googleapis.github.io/python-genai/genai.html#genai.client.Client)

In [5]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION
)

### Helper function

In [9]:
def generate_gemini_2_0(query:str, yt_uri:str) -> GenerateContentResponse:

  # https://googleapis.github.io/python-genai/genai.html#genai.types.Part
  # Check class methods. (from_bytes, from_code_execution_result, from_executable_code, from_function_call, etc... )

  query_text = Part.from_text(query)

  yt_video1 = Part.from_uri(
        file_uri=yt_uri,
        mime_type="video/*",
    )

  contents = [
    Content(
      role="user",
      parts=[
        query_text,
        yt_video1,
      ]
    )
  ]

  # https://googleapis.github.io/python-genai/genai.html#genai.types.SafetySettingDict
  safety_settings = [SafetySetting(category="HARM_CATEGORY_HATE_SPEECH",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_HARASSMENT",threshold="OFF")]


  google_search_tool = Tool(google_search=GoogleSearch())
  tools=[google_search_tool]


  # https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
  generate_content_config = GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = safety_settings,
    tools = tools
  )

  # https://googleapis.github.io/python-genai/genai.html#genai.models.AsyncModels.generate_content_stream
  responses = client.models.generate_content_stream(
    model = MODEL_NAME,
    contents = contents,
    config = generate_content_config,
  )

  return responses

In [16]:

yt_uri="https://www.youtube.com/watch?v=BxEs76gaULE"


prompt = """
        당신은 Youtube를 보고 요리에 대한 내용을 설명해주는 AI Assistant 입니다.
        아래 내용에 관점에 대해서 상세하게 알려주세요.

        1. 요리에 사용된 레시피에 대한 설명
        2. 요리 재료에 대한 설명(영양정보 포함)
        3. 요리 준비시간.
        4. 요리 시간.

"""


responses = generate_gemini_2_0(prompt,yt_uri )

for chunk in responses:
  print(chunk.text, end="")


네, 유튜브 영상 내용을 바탕으로 요리 정보를 자세히 설명해 드리겠습니다.

### 1. 요리 레시피 설명
이 영상에서는 새송이버섯을 이용한 간단한 요리법을 소개합니다. 새송이버섯을 먹기 좋은 크기로 자른 후, 칼집을 내어 양념이 잘 배도록 하는 것이 특징입니다. 마지막으로 버터와 양념을 추가하여 구워주면 맛있는 버섯 요리가 완성됩니다.

#### 요리 순서
1. 새송이버섯 밑동을 자른 후, 3~4등분으로 썰어줍니다.
2. 썰어놓은 버섯에 격자로 칼집을 내어줍니다. 칼집은 양념이 잘 배도록 살짝 깊게 넣어줍니다.
3. 예열된 팬에 식용유 1T를 두르고 버터 2T를 넣어서 녹여줍니다. 버터가 없다면 식용유만 사용해도 괜찮습니다.
4. 버터가 녹으면 칼집을 낸 새송이버섯을 팬에 올립니다.
5. 중약불에서 버섯을 앞뒤로 노릇하게 구워줍니다. 다 구워진 버섯은 팬 한쪽으로 밀어 놓고,
6. 다진 마늘과 송송 썬 쪽파를 넣고 볶아줍니다.
7. 마늘이 살짝 갈색빛을 띄고 향이 올라오면 간장 1T, 굴소스 1/2T, 올리고당 1/2T를 넣고 잘 섞어줍니다.
8. 마지막으로 버섯과 양념을 함께 볶아주다가 양념이 잘 배면 맛술 또는 미림 1T를 추가합니다.
9. 잘 볶아진 버섯은 접시에 담아 맛있게 먹습니다.

### 2. 요리 재료 설명

*   **새송이버섯**:
    *   **영양 정보**: 새송이버섯은 저칼로리, 저지방 식품으로 식이섬유가 풍부하며 비타민 B, 비타민 D, 칼륨 등을 함유하고 있습니다. 특히 식이섬유는 장 건강에 도움을 주고, 비타민 D는 칼슘 흡수를 촉진합니다.
    *   **특징**: 쫄깃한 식감과 담백한 맛이 특징이며, 다양한 요리에 활용됩니다.
*   **마늘**:
    *   **영양 정보**: 알리신이라는 성분을 함유하고 있어 항균, 항암 작용에 도움을 주며, 혈액순환 개선 효과도 있습니다. 또한, 비타민 B와 비타민 C도 함유하고 있습니다.
    *   **특징**: 특유의 매운맛과 향이 있어 요리에 풍미를 더해줍니다.
*   *